<a href="https://colab.research.google.com/github/YvYh/FluxWeb/blob/main/Model_Controverses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BQ**: Get/Load Data

In [3]:
from google.cloud import bigquery
json_path = 'poc-bigdata.json'
bigquery_client = bigquery.Client.from_service_account_json(json_path)

In [4]:
def get_bq_data(query):
  query_job = bigquery_client.query(query)
  rows = query_job.result()
  data = rows.to_dataframe()
  return data

In [6]:
def bq_load_df(name, df):
    dataset_ref = bigquery_client.dataset('FluxWeb_Prediction')
    table_ref = dataset_ref.table(name)
    
    
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
    if name == 'Controverses_bert':
      job_config.schema = get_bigquery_schema()
    else:
      job_config.autodetect=True
 
    load_job = bigquery_client.load_table_from_dataframe(
        df,
        table_ref,
        job_config=job_config)
 
    assert load_job.job_type == 'load'
 
    load_job.result()  # Waits for table load to complete.
 
    assert load_job.state == 'DONE'
    print('table {} load {} data.'.format(name, len(df)))

# Bert


In [ ]:
!pip install -q tensorflow-text
!pip install -q tf-models-official

     |████████████████████████████████| 3.4MB 5.9MB/s 
     |████████████████████████████████| 1.1MB 5.8MB/s 
     |████████████████████████████████| 174kB 47.2MB/s 
     |████████████████████████████████| 358kB 41.5MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 37.6MB 96kB/s 
     |████████████████████████████████| 706kB 40.3MB/s 
     |████████████████████████████████| 102kB 9.9MB/s 
     |████████████████████████████████| 645kB 38.1MB/s 
     |████████████████████████████████| 1.2MB 34.7MB/s 


In [ ]:
!pip install --upgrade scikit-learn

     |████████████████████████████████| 22.3MB 2.0MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
tf.get_logger().setLevel('ERROR')

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3'
 
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [ ]:
def bert_vectorize(text_test):
    
    text_preprocessed = bert_preprocess_model([text_test])
    bert_results = bert_model(text_preprocessed)
    vect = bert_results["pooled_output"][0,:]
    return vect.numpy()
    #return bert_vectorize

In [ ]:
from IPython.display import HTML, display
import time
 
def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [ ]:
q_news = """SELECT distinct id_news,News, NumControverse 
FROM `poc-bigdata.FluxWeb_Prediction.News_test` 
Order by id_news desc
"""
news = get_bq_data(q_news)
news.head()

,id_news,News,NumControverse
0,N957069,Carrefour mise un milliard sur les anciens act...,3265
1,N957069,Carrefour mise un milliard sur les anciens act...,1689
2,N957069,Carrefour mise un milliard sur les anciens act...,3006
3,N957069,Carrefour mise un milliard sur les anciens act...,3412
4,N957069,Carrefour mise un milliard sur les anciens act...,2722


In [ ]:
import pickle
svd=pickle.load(open('svd120.pickle', 'rb'))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TruncatedSVD from version 0.23.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
cosine_similarity(svd.transform([news_bert]),svd.transform(controverses.vect_c.to_list()))[0]

array([0.98004867, 0.8753628 , 0.90957654])

In [ ]:
nb = len(news.id_news.unique())
print(str(nb))
i = 0
out = display(progress(0, nb), display_id=True)
for idn in news.id_news.unique():

  news_text = news[news['id_news']==idn]['News'].iloc[0]
  news_bert = bert_vectorize(news_text)
  nums = news[news['id_news']==idn]['NumControverse']
  #print(i/nb)

  result = pd.DataFrame()
  for idc in nums:
    q_vectC = "SELECT NumControverse,Poids,[Embed_"+", Embed_".join([str(x) for x in range(768)])+"] vect_c FROM `poc-bigdata.FluxWeb_Prediction.Controverses_bert` WHERE NumControverse={}".format(str(idc))
    controverses = get_bq_data(q_vectC)
    cos = cosine_similarity([news_bert],controverses.vect_c.to_list())[0]
    cos120 = cosine_similarity(svd.transform([news_bert]),svd.transform(controverses.vect_c.to_list()))[0]
    s = controverses.Poids.sum()
    poids = [x/s for x in controverses.Poids]
    result = result.append({'id_news':idn,
                            'numControverse':idc,
                            'bert_score':sum(cos*poids),
                            'bert_svd120_score':sum(cos120*poids)}
                            ,ignore_index=True)
  bq_load_df_in_gcs('Controverse_News_Score', result)
  out.update(progress(i, nb))
  i += 1

# **TensorFlow**:  TextVectorization

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [9]:
!pip install -U spacy
!python -m spacy download fr_core_news_lg

     |████████████████████████████████| 12.8MB 235kB/s 
     |████████████████████████████████| 624kB 34.2MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
     |████████████████████████████████| 460kB 35.9MB/s 
     |████████████████████████████████| 9.1MB 41.4MB/s 
     |████████████████████████████████| 122kB 49.4MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107107 sha256=2e151a92a72bc517c271b23536f0c7aef8c33534f37b064d29634e0d96f0d6ad
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found e

In [64]:
def text_preprossing(text):
    import spacy
    nlp = spacy.load("fr_core_news_lg")
    tokens = nlp(text)
    clean = []
    for token in tokens:
        if not any([token.is_digit, token.is_punct, token.is_currency,
               token.like_url, token.like_num, token.like_email,
               token.is_stop]):
            clean += [token.lemma_]
    return tf.strings.join(clean, separator=' ')

In [65]:
vocab_size = 10000
sequence_length = 250

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Set output_sequence_length length to pad all samples to same length.
vectorize_layer = TextVectorization(
    standardize= text_preprossing,#'lower_and_strip_punctuation'
    max_tokens=vocab_size,
    output_mode='tf-idf',
    #output_sequence_length=sequence_length
    )

In [ ]:
q = """SELECT DISTINCT NumControverse as label, Titre as Text
FROM `poc-bigdata.FluxWeb_Prediction.Controverses_prd`
UNION ALL 
SELECT DISTINCT NumControverse as label, Commentaire as Text
FROM `poc-bigdata.FluxWeb_Prediction.Controverses_prd`
WHERE Commentaire is not null and length(Commentaire)>5
UNION ALL 
SELECT DISTINCT NumControverse as label, Informations as Text
FROM `poc-bigdata.FluxWeb_Prediction.Controverses_prd`
WHERE Informations is not null
"""
controverses = get_bq_data(q)

In [52]:
controverses.head()

,label,Text
0,2866,"Le 8 mai 2019, FIN24 a annoncé que l'ancien di..."
1,2866,Steinhoff International Holdings a annoncé que...
2,2866,Suite à un rapport de PWC révélant que des dir...
3,389,EDF a annoncé que les échanges avec l'Autorité...
4,389,"EDF a détecté, à partir du 21 mars 2018, des é..."


In [43]:
dataset = tf.data.Dataset.from_tensor_slices((controverses.Text.values, controverses.label.values))
dataset

<TensorSliceDataset shapes: ((), ()), types: (tf.string, tf.int64)>

In [49]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(controverses, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

2400 train examples
600 validation examples
751 test examples


In [58]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('label')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [59]:
batch_size = 32 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)
train_ds

<BatchDataset shapes: ({Text: (None,)}, (None,)), types: ({Text: tf.string}, tf.int64)>

In [60]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  #print('A batch of ages:', feature_batch['Age'])
  print('A batch of targets:', label_batch )

Every feature: ['Text']
A batch of targets: tf.Tensor(
[3102 2995 2788 3149 2689 2476 3254 2776 2470 3320 1495 2806 2546  453
 2747 3351 2586 2833 2478 2919 2631 1361 2222 3353 1103 2851 3374 2658
 2457 2604 2411 2869], shape=(32,), dtype=int64)


In [66]:
header='Text'
all_inputs = [tf.keras.Input(shape=(1,), name=header)]

In [67]:
vectorize_layer.adapt(train_ds.map(lambda x, y: x[header]))
encoded_features = [vectorize_layer]

TypeError: ignored

In [ ]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(32, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(all_inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [ ]:
# rankdir='LR' is used to make the graph horizontal.
tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

In [ ]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

In [ ]:
model.save('my_pet_classifier')
reloaded_model = tf.keras.models.load_model('my_pet_classifier')

In [ ]:
news =''

input_dict = {'Text': tf.convert_to_tensor([news])}
predictions = reloaded_model.predict(input_dict)
prob = tf.nn.sigmoid(predictions[0])

print(
    "This particular pet had a %.1f percent probability "
    "of getting adopted." % (100 * prob)
)